In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from math import radians

In [51]:
pref = 'financiero'

In [27]:
df1 = pd.read_csv('reto_precios.csv')
df2 = pd.read_csv('financieros.csv', encoding='latin-1')

C:\Users\alber\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
df2.columns

Index(['id', 'clee', 'nom_estab', 'raz_social', 'codigo_act', 'nombre_act',
       'per_ocu', 'tipo_vial', 'nom_vial', 'tipo_v_e_1', 'nom_v_e_1',
       'tipo_v_e_2', 'nom_v_e_2', 'tipo_v_e_3', 'nom_v_e_3', 'numero_ext',
       'letra_ext', 'edificio', 'edificio_e', 'numero_int', 'letra_int',
       'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom', 'num_local',
       'cod_postal', 'cve_ent', 'entidad', 'cve_mun', 'municipio', 'cve_loc',
       'localidad', 'ageb', 'manzana', 'telefono', 'correoelec', 'www',
       'tipoUniEco', 'latitud', 'longitud', 'fecha_alta'],
      dtype='object')

In [29]:
df1 = df1[['id','lat','lon']]
# Cambiar los nombres de las variables en df1 agregando sufijo 1
df1 = df1.rename(columns={ 'lat': 'lat1', 'lon': 'lon1'})
# Cambiar los nombres de las variables en df2 agregando sufijo 2
df2 = df2.rename(columns={'id': 'id2', 'latitud': 'lat2', 'longitud': 'lon2'})


In [30]:
# Definir la tolerancia espacial (en kilómetros)
tolerancia = 0.5

# Convertir las latitudes y longitudes a radianes
df1['lat_rad1'] = df1['lat1'].apply(radians)
df1['lon_rad1'] = df1['lon1'].apply(radians)
df2['lat_rad2'] = df2['lat2'].apply(radians)
df2['lon_rad2'] = df2['lon2'].apply(radians)

In [31]:
# Calcular la distancia haversine entre las coordenadas
distances = haversine_distances(df1[['lat_rad1', 'lon_rad1']], df2[['lat_rad2', 'lon_rad2']]) * 6371000/1000  # Convertir a kilómetros

# Encontrar las coincidencias dentro de la tolerancia
matches = distances <= tolerancia

In [32]:
matching_indices = np.where(matches)

# Crear una lista para almacenar los DataFrames combinados
combined_dfs = []

# Iterar sobre los índices y columnas de coincidencias
for idx, col in zip(*matching_indices):
    # Obtener los registros coincidentes de df1 y df2
    matching_record_df1 = df1.loc[idx]
    matching_record_df2 = df2.loc[col]
    
    # Combinar los registros en un DataFrame
    combined_record = pd.concat([matching_record_df1, matching_record_df2])
    combined_dfs.append(combined_record)

# Concatenar todos los DataFrames combinados en uno solo
final_combined_df = pd.concat(combined_dfs, axis=1).T

In [33]:
final_combined_df.head(3)

,id,lat1,lon1,lat_rad1,lon_rad1,id2,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,tipo_v_e_2,nom_v_e_2,tipo_v_e_3,nom_v_e_3,numero_ext,letra_ext,edificio,edificio_e,numero_int,letra_int,tipo_asent,nomb_asent,tipoCenCom,nom_CenCom,num_local,cod_postal,cve_ent,entidad,cve_mun,municipio,cve_loc,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,lat2,lon2,fecha_alta,lat_rad2,lon_rad2
0,1.0,19.421317,-99.164182,0.338966,-1.730741,6896560,09015522110014741000000000U8,AFIRME,BANCA AFIRME,522110,Banca múltiple,0 a 5 personas,AVENIDA,INSURGENTES ESQ. URUAPAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3959.0,NaN,COLONIA,ROMA NORTE,NaN,NaN,NaN,6700.0,9,CIUDAD DE MÉXICO,15,Cuauhtémoc,1,Cuauhtémoc,0998,13,NaN,NaN,NaN,Fijo,19.421028,-99.163889,2019-04,0.338961,-1.730736
1,1.0,19.421317,-99.164182,0.338966,-1.730741,8326800,09015524210002792000000000U1,AGENCIA DE FIANZAS & SEGUROS DESPACHO CAMPANELLA,NaN,524210,"Agentes, ajustadores y gestores de seguros y f...",6 a 10 personas,CALLE,SINALOA,CALLE,3 PONIENTE (MEDELLIN),CALLE,VALLADOLID,CALLE,DURANGO,84.0,NaN,NaN,NaN,33.0,NaN,COLONIA,ROMA NORTE,NaN,NaN,NaN,6700.0,9,CIUDAD DE MÉXICO,15,Cuauhtémoc,1,Cuauhtémoc,0998,4,NaN,NaN,WWW.CAMPANELLA.COM.MX,Fijo,19.420569,-99.167557,2019-11,0.338953,-1.7308
2,1.0,19.421317,-99.164182,0.338966,-1.730741,8386346,09015522460000161000000000S4,ALRA DEL SUR SUCURSAL CUAUHTEMOC,ALRA DEL SUR SUCURSAL CUAHUTEMOC SA DE CV SOFO...,522460,Sociedades financieras de objeto múltiple,0 a 5 personas,CALLE,SINALOA,EJE VIAL,3 PONIENTE (SALAMANCA),CALLE,VALLADOLID,CALLE,DURANGO,106.0,NaN,EDIFICIO DE OFCINAS,PISO 10,1001.0,NaN,COLONIA,ROMA NORTE,EDIFICIO COMERCIAL Y DE SERVICIOS,EDIFICIO DE OFICINAS,1001,6700.0,9,CIUDAD DE MÉXICO,15,Cuauhtémoc,1,Cuauhtémoc,0998,5,NaN,ALRADELSUR@CIVA.MX,CREDITOFOVISSSTE.NET,Fijo,19.420307,-99.168741,2019-11,0.338948,-1.730821


In [34]:
len(final_combined_df)

54100

In [35]:
final_combined_df['numero_ext'].value_counts()

42.0     889
81.0     554
236.0    510
153.0    459
287.0    441
        ... 
574.0      1
48.0       1
64.0       1
213.0      1
306.0      1
Name: numero_ext, Length: 267, dtype: int64

In [36]:
final_combined_df['manzana'].value_counts()

2     5011
8     4837
6     3573
10    2986
9     2900
18    2851
17    2494
4     2479
16    2384
14    2344
12    2137
3     1959
13    1699
1     1692
25    1567
20    1565
15    1532
7     1517
11    1205
5     1180
19    1179
24    1115
21    1023
23     976
28     672
22     496
27     319
29     138
26     127
46      63
30      27
37      22
40      12
42       7
33       6
49       2
31       2
34       1
32       1
Name: manzana, dtype: int64

In [37]:
final_combined_df['per_ocu'].value_counts()

0 a 5 personas        39497
6 a 10 personas        8720
11 a 30 personas       3879
31 a 50 personas        656
251 y más personas      503
51 a 100 personas       473
101 a 250 personas      372
Name: per_ocu, dtype: int64

In [38]:
personal_ocupado = {'0 a 5 personas' : 1,
'6 a 10 personas' : 2,
'11 a 30 personas' : 3,
'31 a 50 personas' : 4,
'51 a 100 personas' : 5,
'101 a 250 personas' : 6,
'251 y más personas' : 7}

final_combined_df['per_ocu'] = final_combined_df['per_ocu'].replace(personal_ocupado)
final_combined_df['per_ocu'].value_counts()

1    39497
2     8720
3     3879
4      656
7      503
5      473
6      372
Name: per_ocu, dtype: int64

In [39]:
#per_ocu
#numero_ext
#manzana

In [40]:
final_combined_df['dif_lat_rad'] =final_combined_df['lat_rad1']-final_combined_df['lat_rad2']
final_combined_df['dif_lon_rad'] =final_combined_df['lon_rad1']-final_combined_df['lon_rad2']

In [41]:
(final_combined_df['lat_rad1']-final_combined_df['lat_rad2'])

0        0.000005
1        0.000013
2        0.000018
3        -0.00006
4             0.0
           ...   
54095    -0.00005
54096   -0.000031
54097    0.000031
54098    -0.00003
54099   -0.000052
Length: 54100, dtype: object

In [42]:
final_combined_df.entidad.value_counts()

CIUDAD DE MÉXICO    53200
NUEVO LEÓN            899
BAJA CALIFORNIA         1
Name: entidad, dtype: int64

In [43]:
final_combined_df[final_combined_df.entidad.str.contains('NUEVO LEÓN')][['id','dif_lat_rad','dif_lon_rad']]

,id,dif_lat_rad,dif_lon_rad
2909,54.0,0.000005,0.000049
2910,54.0,-0.000022,0.000009
2911,54.0,0.000035,-0.000023
2912,54.0,0.000059,-0.000021
2913,54.0,0.000047,-0.000013
...,...,...,...
52239,952.0,0.000025,-0.000073
52240,952.0,0.000035,-0.000066
52241,952.0,0.000025,-0.000073
52242,952.0,0.000003,-0.000067


In [44]:
final_combined_df[final_combined_df.entidad.str.contains('BAJA CALIFORNIA')][['id','dif_lat_rad','dif_lon_rad']]

,id,dif_lat_rad,dif_lon_rad
32198,583.0,0.000017,-0.000075


In [45]:
summary = final_combined_df[['id','per_ocu','numero_ext','manzana']]
summary.head()

,id,per_ocu,numero_ext,manzana
0,1.0,1,NaN,13
1,1.0,2,84.0,4
2,1.0,1,106.0,5
3,1.0,2,31.0,19
4,1.0,1,NaN,17


In [60]:
# Definir las columnas para la agrupación y agregación
aggregation_columns = ['per_ocu', 'numero_ext', 'manzana']  # Agrega aquí las columnas que quieras sumar

# Realizar la agrupación y las agregaciones
grouped_df = summary.groupby('id')[aggregation_columns].agg({'sum'}).reset_index()
grouped_df2 = summary.groupby('id').agg(total=('id', 'count')).reset_index()
grouped_df2.columns = ['id'] + [f'{pref}_total']



# Renombrar las columnas resultantes
grouped_df.columns = ['id'] + [f'{pref}_suma_{col}' for col in aggregation_columns]

# Mostrar el DataFrame resultante
centros = grouped_df2.merge(grouped_df)

In [61]:
centros.head()

,id,financiero_total,financiero_suma_per_ocu,financiero_suma_numero_ext,financiero_suma_manzana
0,1.0,111,161,6896.0,1052
1,2.0,101,149,6182.0,947
2,3.0,22,33,2183.0,244
3,4.0,18,22,1895.0,189
4,5.0,43,67,1619.0,628


In [62]:
centros.to_csv('centros_'+pref+'s'+'.csv')